# BLEU

https://github.com/moses-smt/mosesdecoder/blob/master/scripts/generic/multi-bleu.perl

## Results

From: <a href="../paper/webnlg2017/The WebNLG Challenge -  Generating Text from RDF Data.pdf">The WebNLG Challenge - Generating Text from RDF Data</a>

<img src='../img/bleu_result.jpg' />

## Melbourne

<a href="../data/webnlg2017/submissions/melbourne/WebNLG Challenge - Bayu Distiawan T.pdf">report</a>

In [1]:
!../evaluation/webnlg2017/webnlg-baseline-master/multi-bleu.perl -lc ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference0.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference1.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference2.lex < ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/GKB_Unimelb_all-cat.txt

BLEU = 45.13, 79.9/55.4/38.8/27.4 (BP=0.969, ratio=0.969, hyp_len=40642, ref_len=41921)


In [3]:
!head -100 ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/GKB_Unimelb_all-cat.txt | tail -10

castle is english language .
eric flint was born in burbank , california .
macmillan publishers is the parent company of farrar , straus and giroux .
john cowper powys was a glastonbury romance .
soho press is in united states .
the secret scripture was published by faber and faber .
asian americans are an ethnic group in united states .
english language is spoken in united states .
weymouth sands was preceded by a glastonbury romance .
the manager of a . c . chievo verona is rolando maran .


## PKUWriter

<a href="../data/webnlg2017/submissions/pkuwriter/PKUWriter_system.pdf">report</a>

In [2]:
!head -100 ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/PKUWriter_all-cat.txt | tail -10

the novel into battle is written by the english language .
john cowper powys was born in burbank , california .
the parent company of the macmillan publishers press is the macmillan publishers company of the macmillan publishers .
the novel owen glendower is a notable work by john cowper powys .
soho press is located in united states .
the polish academy for writtenwork is referencenumber _ in _ the _ national _ register _ of _ historic _ places .
asian americans are an ethnic group in the united states .
english is the language spoken in the united states .
the novel into battle is preceded by a glastonbury romance .
rolando maran manages the a . c . chievo verona .


In [5]:
!../evaluation/webnlg2017/webnlg-baseline-master/multi-bleu.perl -lc ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference0.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference1.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference2.lex < ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/PKUWriter_all-cat.txt

BLEU = 39.88, 73.9/49.9/35.6/26.0 (BP=0.927, ratio=0.930, hyp_len=38248, ref_len=41143)


## UPF-FORGe

<a href="../data/webnlg2017/submissions/upf/WebNLG_V0.2.pdf">report</a>

In [4]:
!head -100 ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/UPF-TALN_all-cat.txt | tail -10

the english language is spoken in castle ( novel ) .
eric flint was born in burbank ( california ) .
the parent company of farrar , straus and giroux is macmillan publishers .
a glastonbury romance wrote john cowper powys .
soho press is published in the united states .
the publisher of the secret scripture is faber and faber .
a ethnic group of the united states are asian americans .
the english language is spoken in the united states .
weymouth sands follows a glastonbury romance .
rolando maran is the manager of a . c . chievo verona .


In [6]:
!../evaluation/webnlg2017/webnlg-baseline-master/multi-bleu.perl -lc ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference0.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference1.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference2.lex < ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/UPF-TALN_all-cat.txt

BLEU = 38.65, 75.5/48.7/31.1/19.5 (BP=1.000, ratio=1.064, hyp_len=47234, ref_len=44405)


## Tilb-Pipeline

<a href="../data/webnlg2017/submissions/tilburg/report.pdf">report</a>

In [5]:
!head -100 ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/Tilburg_University-3_all-cat.txt | tail -10

castle is in english .
eric flint was born in burbank , california .
macmillan publishers owns farrar , straus , and giroux .
a glastonbury romance is a notable work by john cowper powys .
soho press is in united states .
the secret scripture publisher of faber and faber .
asian americans live in united states .
united states is in english .
weymouth sands was preceded by a glastonbury romance .
rolando maran manages a . c . chievo verona .


In [8]:
!../evaluation/webnlg2017/webnlg-baseline-master/multi-bleu.perl -lc ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference0.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference1.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference2.lex < ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/Tilburg_University-3_all-cat.txt

BLEU = 35.29, 81.8/56.6/38.8/26.5 (BP=0.755, ratio=0.781, hyp_len=30335, ref_len=38854)


## Tilb-NMT

<a href="../data/webnlg2017/submissions/tilburg/report.pdf">report</a>

In [6]:
!head -100 ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/Tilburg_University-1_all-cat.txt | tail -10

english is the language of castle .
eric flint was born in burbank , california .
macmillan publishers is the parent company of farrar , straus , and giroux .
a glastonbury romance is a notable work by john cowper powys .
soho press is located in united states .
th secret scriptures was published by faber and faber .
asian americans are an ethnic group in united states .
english is the language of united states .
weymouth sands was preceded by a glastonbury romance .
the manager of a . c . chievo verona is rolando maran .


In [9]:
!../evaluation/webnlg2017/webnlg-baseline-master/multi-bleu.perl -lc ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference0.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference1.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference2.lex < ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/Tilburg_University-1_all-cat.txt

BLEU = 34.60, 68.8/42.7/27.5/17.7 (BP=1.000, ratio=1.054, hyp_len=45605, ref_len=43278)


## Baseline

In [7]:
!head -100 ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/Baseline_all-cat.txt | tail -10

the english language is spoken in the republic of ireland .
author was born in burbank , california .
the parent company of wiley - blackwell is macmillan publishers .
a glastonbury romance was written by author .
lahore is in the united states .
the parent company of wiley - blackwell is parentcompany .
asian americans are an ethnic group in the united states .
english is the language of the united states .
a glastonbury romance was preceded by precededby .
the manager of a . c . chievo verona is rolando maran .


In [10]:
!../evaluation/webnlg2017/webnlg-baseline-master/multi-bleu.perl -lc ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference0.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference1.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference2.lex < ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/Baseline_all-cat.txt

BLEU = 33.24, 65.4/42.3/31.8/24.8 (BP=0.865, ratio=0.873, hyp_len=35312, ref_len=40435)


## Adapt

<a href="../data/webnlg2017/submissions/adaptCenter/WebNLG_system.pdf">report</a>

In [8]:
!head -100 ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/ADAPT_Centre_all-cat.txt | tail -10

the novel castle is the language spoken in the english language .
eric flint was born in burbank , california .
macmillan publishers is the parent company of farrar , straus and giroux .
a glastonbury romance is the notable work of john cowper powys .
soho press is located in the united states .
the novel the secret scriptures , is published by faber and faber .
asian americans are an ethnic group in the u . s .
english is the language of the united states .
a glastonbury romance is the sequel to weymouth sands .
the manager of a . c . chievo verona is rolando maran .


In [11]:
!../evaluation/webnlg2017/webnlg-baseline-master/multi-bleu.perl -lc ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference0.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference1.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference2.lex < ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/ADAPT_Centre_all-cat.txt

BLEU = 31.06, 54.9/35.7/25.3/18.7 (BP=1.000, ratio=1.337, hyp_len=59727, ref_len=44663)


## UIT-VNU

<a href="../data/webnlg2017/submissions/uit-danglt-clnlp/Phong-Nguyen-Tran_Dang-Tuan-Nguyen.docx">report</a>

In [9]:
!head -100 ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/UIT-DANGNT-CLNLP_all-cat.txt | tail -10

english language is one of the languages that is spoken in castle ( novel ) .
the birth place of eric flint is burbank , california .
macmillan publishers is the parent company of the farrar , straus and giroux .
null
soho press is located within the country of the united states .
the book the secret scripture was published by faber and faber .
asian americans are part of one of the ethnic groups in the united states .
english language is one of the languages that is spoken in united states .
a glastonbury romance was written prior to weymouth sands .
rolando maran is the manager of the a . c . chievo verona .


In [12]:
!../evaluation/webnlg2017/webnlg-baseline-master/multi-bleu.perl -lc ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference0.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference1.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference2.lex < ../evaluation/webnlg2017/webnlg-automatic-evaluation/teams/UIT-DANGNT-CLNLP_all-cat.txt

BLEU = 7.07, 67.5/54.4/39.5/29.2 (BP=0.156, ratio=0.350, hyp_len=11219, ref_len=32072)


# My models

In [34]:
models = !ls -R -d ../tmp/**/*

models = [m for m in models if m.endswith('_all-cat.txt')]

In [37]:
for model in models:
    
    print(model)
    
    !../evaluation/webnlg2017/webnlg-baseline-master/multi-bleu.perl -lc ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference0.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference1.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference2.lex < "$model"
    
    print()

../tmp/4 - Baseline - Just join triples_e089e6d2328637218601e68ee1216f624e70d78f/4 - Baseline - Just join triples_e089e6d2328637218601e68ee1216f624e70d78f_all-cat.txt
BLEU = 23.56, 61.2/34.8/19.6/11.4 (BP=0.896, ratio=0.901, hyp_len=36820, ref_len=40883)

../tmp/5 - Model - Template Based - roots_3ae48a5d03a715d609dc8d73f82ac5682cba128d/5 - Model - Template Based - roots_3ae48a5d03a715d609dc8d73f82ac5682cba128d_all-cat.txt
BLEU = 36.88, 74.6/46.6/29.2/18.3 (BP=1.000, ratio=1.044, hyp_len=46087, ref_len=44165)

../tmp/5 - Model - Template Based - roots_4e9245cbbc05e4fdf0de404fbada20d8294127b2/5 - Model - Template Based - roots_4e9245cbbc05e4fdf0de404fbada20d8294127b2_all-cat.txt
BLEU = 40.12, 75.6/49.0/32.5/21.5 (BP=1.000, ratio=1.037, hyp_len=45726, ref_len=44096)

../tmp/5 - Model - Template Based - roots_55f9d82b420c5cbc6a01d5a65082414b87a81195/5 - Model - Template Based - roots_55f9d82b420c5cbc6a01d5a65082414b87a81195_all-cat.txt
BLEU = 39.97, 74.8/48.9/32.5/21.5 (BP=1.000, ratio=1.